In [5]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding

In [7]:
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [8]:
from nltk.corpus import wordnet
nltk.download('tagsets')
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\keert\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


In [9]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = word[1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [10]:
class SimpleTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
        self.stopwords = set(ENGLISH_STOP_WORDS)
        self.lemmatizer = WordNetLemmatizer()


    def transform(self, x, fit=False):
        matrix = []
        i = 0;
        for sentence in x:
            tokens = self.tokenizer.tokenize(sentence.lower())
            tokens_tags = nltk.pos_tag(tokens)   
            tokens = [self.lemmatizer.lemmatize(item[0], get_wordnet_pos(item)) for item in tokens_tags if item[1] not in ['MD','IN', 'FW', 'EX','PDT','POS', 'PRP$','PRP', 'UH', 'WDT', 'WRB','WP','WP$']]  
            tokens = [word for word in tokens if word not in self.stopwords and len(word) > 1]
            
            matrix.append(tokens)
            
            if i % 50 == 0:
                print(i)
            i = i + 1
                              
        return matrix

nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\keert\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\keert\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\keert\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\omw-1.4.zip.


True

In [13]:
df = pd.read_csv('movieplotswiki_movie_plots_deduped.csv')


In [14]:
pd.set_option('display.max_columns',None)

df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [15]:
df.dropna(inplace=True)

df['final'] = df['Genre'].astype('category').cat.codes


df.drop(['Genre', 'Wiki Page', 'Cast','Director', 'Origin/Ethnicity', 'Title', 'Release Year'], axis = 1, inplace=True)

df_target = df['final'].to_numpy()

df1 = df.to_numpy()

df1 = df1.tolist()

In [16]:
df_2 = []

for a in df1:
    df_2.append(a[0])

In [17]:
tokenzier = SimpleTransformer()
tokens = tokenzier.transform(df_2)
tokens[0]


0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050
10100
10150

['alice',
 'follow',
 'large',
 'white',
 'rabbit',
 'rabbit',
 'hole',
 'tiny',
 'door',
 'bottle',
 'label',
 'drink',
 'shrink',
 'pass',
 'door',
 'eat',
 'labeled',
 'eat',
 'grows',
 'large',
 'fan',
 'enable',
 'shrink',
 'garden',
 'try',
 'dog',
 'play',
 'enter',
 'white',
 'rabbit',
 'tiny',
 'house',
 'suddenly',
 'resume',
 'normal',
 'size',
 'order',
 'use',
 'magic',
 'fan',
 'enter',
 'kitchen',
 'cook',
 'woman',
 'hold',
 'baby',
 'persuade',
 'woman',
 'child',
 'infant',
 'cook',
 'start',
 'throw',
 'thing',
 'baby',
 'turn',
 'pig',
 'squirm',
 'grip',
 'duchess',
 'cheshire',
 'cat',
 'appear',
 'disappear',
 'couple',
 'time',
 'alice',
 'direct',
 'mad',
 'hatter',
 'mad',
 'tea',
 'party',
 'leave',
 'queen',
 'invite',
 'alice',
 'join',
 'royal',
 'procession',
 'parade',
 'march',
 'playing',
 'card',
 'head',
 'white',
 'rabbit',
 'alice',
 'unintentionally',
 'offend',
 'queen',
 'summons',
 'executioner',
 'alice',
 'box',
 'ear',
 'flees',
 'playing',


In [18]:
from nltk import FreqDist
fdist = FreqDist(np.concatenate(tokens)) 
for word, frequency in fdist.most_common(2000):
    print(u'{};{}'.format(word, frequency))

kill;32247
tell;28628
leave;28500
love;23317
father;23245
make;23128
try;20907
friend;20076
return;19967
come;19754
life;18477
day;18232
time;18173
family;18070
meet;18068
man;17788
home;17338
house;16802
help;16599
police;16317
new;16267
later;16030
mother;14991
work;14891
year;14377
son;14336
wife;14298
end;14217
film;14109
fall;13813
use;13490
know;13402
young;13164
marry;13114
escape;12761
begin;12607
woman;12590
want;12586
daughter;12279
away;12270
run;12247
ask;12145
money;11759
attempt;11739
night;11591
brother;11516
death;11512
girl;11510
child;11424
plan;11382
decide;11093
old;11082
say;11055
turn;11006
reveal;10575
start;10527
arrive;10512
way;10483
car;10424
men;10326
force;10303
live;10102
story;9983
fight;9881
lead;9856
attack;9827
order;9694
murder;9447
die;9274
shoot;9113
discover;8691
learn;8652
play;8560
send;8511
realize;8476
soon;8451
follow;8444
save;8442
school;8386
just;8302
break;8254
head;8193
set;8110
sister;7865
place;7815
bring;7803
believe;7775
visit;7752
re

In [19]:
fdist.most_common(10_000)

np.array(tokens).flatten()[0:2]


<ipython-input-19-11d9b2d8b918>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(tokens).flatten()[0:2]


array([list(['alice', 'follow', 'large', 'white', 'rabbit', 'rabbit', 'hole', 'tiny', 'door', 'bottle', 'label', 'drink', 'shrink', 'pass', 'door', 'eat', 'labeled', 'eat', 'grows', 'large', 'fan', 'enable', 'shrink', 'garden', 'try', 'dog', 'play', 'enter', 'white', 'rabbit', 'tiny', 'house', 'suddenly', 'resume', 'normal', 'size', 'order', 'use', 'magic', 'fan', 'enter', 'kitchen', 'cook', 'woman', 'hold', 'baby', 'persuade', 'woman', 'child', 'infant', 'cook', 'start', 'throw', 'thing', 'baby', 'turn', 'pig', 'squirm', 'grip', 'duchess', 'cheshire', 'cat', 'appear', 'disappear', 'couple', 'time', 'alice', 'direct', 'mad', 'hatter', 'mad', 'tea', 'party', 'leave', 'queen', 'invite', 'alice', 'join', 'royal', 'procession', 'parade', 'march', 'playing', 'card', 'head', 'white', 'rabbit', 'alice', 'unintentionally', 'offend', 'queen', 'summons', 'executioner', 'alice', 'box', 'ear', 'flees', 'playing', 'card', 'come', 'wake', 'realize', 'dream']),
       list(['boone', 'daughter', 'befr

In [20]:
unique_words = set([j for sub in tokens for j in sub])
len(unique_words)


117877

In [21]:
import sklearn 
X, test, y, y_test = sklearn.model_selection.train_test_split(tokens, df_target, test_size=.15)


pd.Series(y).value_counts()

839     4927
2117    4591
554     3705
1357     968
7        936
        ... 
1846       1
1830       1
1782       1
1750       1
1432       1
Length: 1985, dtype: int64

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=.01, max_df=.99, max_features=10_000, lowercase=False)
X2 = vectorizer.fit_transform([" ".join(x) for x in X])
X2.shape
test2 = vectorizer.transform([" ".join(x) for x in test])


X2.shape

(28444, 2174)

In [23]:
def build_model():  
    model = models.Sequential()
    model.add(layers.Dense(2158, activation='relu', input_shape=(X2.shape[1],)))
    model.add(layers.Dropout(.4))
    model.add(layers.Dense(800, activation='relu'))
    model.add(layers.Dropout(.4))
    model.add(layers.Dense(300, activation='relu'))
    model.add(layers.Dropout(.4))
    model.add(layers.Dense(2002, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[tf.metrics.Recall(), 'accuracy'])      
    return model
model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2158)              4693650   
                                                                 
 dropout (Dropout)           (None, 2158)              0         
                                                                 
 dense_1 (Dense)             (None, 800)               1727200   
                                                                 
 dropout_1 (Dropout)         (None, 800)               0         
                                                                 
 dense_2 (Dense)             (None, 300)               240300    
                                                                 
 dropout_2 (Dropout)         (None, 300)               0         
                                                                 
 dense_3 (Dense)             (None, 2002)              6

In [33]:
weights = dict([(i, r) for i, r in enumerate(sklearn.utils.class_weight.compute_class_weight('balanced', classes=np.unique(y), y = y))])

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier #scikeras.wrappers
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
model = KerasClassifier(build_fn=build_model, epochs=5, batch_size=32, verbose=1, class_weight=weights)
#callbacks = [EarlyStopping(monitor='loss', patience=1,restore_best_weights=True)]
modelfit = model.fit(X2.toarray(), y)

In [ ]:
predictedp = model.predict_proba(test2.toarray())
predicted = model.predict(test2.toarray())


In [31]:
import scikitplot as skplt
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score, f1_score
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
plt.rcParams['figure.figsize'] = [10, 10]



In [ ]:
print(model.score(test2.toarray(), y_test))
            
print(accuracy_score(y_test, predicted))
print(precision_score(y_test, predicted, average='macro'))
print(recall_score(y_test, predicted,average='macro'))
print(f1_score(y_test, predicted,average='macro'))